© 2024 Rebecca J. Rousseau and Justin B. Kinney, *Algebraic and diagrammatic methods for the rule-based modeling of multi-particle complexes*. This work is licensed under a [Creative Commons Attribution License CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/). All code contained herein is licensed under an [MIT license](https://opensource.org/licenses/MIT).
___

# Deterministic simulation for monomer system

In [1]:
import numpy as np
from itertools import combinations
from scipy import sparse, linalg

In [2]:
# Define "dictionary" of mathematical operators tracking field formation (hat), degradation (check), presence (bar),
# and absence (tilde), as well as identity (id) and "zero" (zero) operators.

A_hat_el = sparse.csr_matrix(np.array([[0, 1], [0, 0]]))
A_bar_el = sparse.csr_matrix(np.array([[1, 0], [0, 0]]))
A_tilde_el = sparse.csr_matrix(np.array([[0, 0], [0, 1]]))
A_check_el = sparse.csr_matrix(np.array([[0, 0], [1, 0]]))
I_el = sparse.csr_matrix(np.array([[1, 0], [0, 1]]))
zero_el = sparse.csr_matrix(np.array([[0, 0], [0, 0]]))

In [3]:
def multikron(mat_list):
    """Computes Kronecker product of multiple matrices"""
    n = len(mat_list)
    assert n>0
    out_mat = mat_list[0]
    for mat in mat_list[1:]:
        out_mat = sparse.kron(out_mat, mat, format='csc')
    return out_mat

In [4]:
def op_i(mat, i, N):
    """Converts template matrix to full operator based on index"""
    return multikron([mat if j==i else I_el for j in range(N)])

In [5]:
# Set number of internal indices and rates

N = 20 # Total number of possible monomer internal states
r_A_cre = 1 # Rate of monomer creation
r_A_deg = 1 # Rate of monomer degradation

In [ ]:
# Compute transition operator W

W = multikron([zero_el]*N)
for i in range(N):
    W += r_A_cre*op_i(A_hat_el-A_tilde_el, i, N) + r_A_deg*op_i(A_check_el-A_bar_el, i, N)

In [6]:
# Compute A counting matrix

A_bar = multikron([zero_el]*N)
for i in range(N):
    A_bar += op_i(A_bar_el, i, N)

In [7]:
# Construct ground state vector

ground_el = sparse.csc_matrix(np.array([0, 1]))
ground_state = multikron([ground_el]*N)
ground_state = ground_state.toarray().squeeze()

In [8]:
# Construct |sum> vector to take inner product for calculating expectation values

sum_vec = np.ones(2**N).T

In [9]:
# Compute evolving system state function

from scipy.sparse.linalg import expm_multiply
t_stop = 10
num_timepoints = 2001
psi_array = expm_multiply(W,ground_state.T,start=0.0, stop=t_stop, num=num_timepoints)

In [10]:
# Compute the number of A at times t

A_of_t = np.zeros(num_timepoints)
for t in range(num_timepoints):
    psi_t = psi_array[t,:]
    A_of_t[t] = sum_vec.dot(A_bar.dot(psi_t))

In [12]:
folderpath = "../simulationdata"
np.savetxt(f"{folderpath}/A_of_t_monomer_N{N}.csv",A_of_t,delimiter=",")